In [9]:
## Importing libraries
import pandas as pd
import numpy as np
import os


In [10]:
## Setting project home
proj_home = '/home/smaharana1/sec_sentiment'
os.chdir(proj_home)

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

## File path
hadoop fs -copyFromLocal /home/smaharana1/sec_sentiment/data/interim/17k_11_25_2018_21_16.csv /data/MSA_805
0_Spring_18/proj09/

In [12]:
df= pd.read_csv('data/interim/17k_11_25_2018_21_16.csv')

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.02)


## Using Pretrained Embeddings(Glove)

https://www.kaggle.com/hamishdickson/bidirectional-lstm-in-keras-with-glove-embeddings

In [14]:
# # Embedding
maxlen = 200
max_features = 1000

In [15]:
embeddings_index = {}
f = open(os.path.join('data/interim', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [16]:
X = train['clean_text']
test_X = test['clean_text']
Y = train['sentiment'].values

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))

X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=maxlen)
test_X = tokenizer.texts_to_sequences(test_X)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=1)

In [18]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 151354 unique tokens.


In [19]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 100

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))


1001


In [20]:

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

In [53]:
# Convolution
kernel_size = 5
filters = 64
pool_size = 4


In [24]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential

In [56]:
from keras.layers import Activation,Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=maxlen,
                    trainable=True))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(32,unroll=True))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 200, 100)          100100    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200, 100)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 196, 64)           32064     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 49, 64)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total para

In [58]:
# Training
batch_size = 250
epochs = 3

In [70]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
Y_val

array([0, 1, 1, ..., 1, 0, 1])

In [72]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, verbose=1)

Train on 13054 samples, validate on 4352 samples
Epoch 1/3
13054/13054 [==============================] - 14s 1ms/step - loss: 8.0103 - acc: 0.4975 - val_loss: 7.9382 - val_acc: 0.5021
Epoch 2/3
13054/13054 [==============================] - 7s 501us/step - loss: 8.0103 - acc: 0.4975 - val_loss: 7.9382 - val_acc: 0.5021
Epoch 3/3
13054/13054 [==============================] - 7s 500us/step - loss: 8.0103 - acc: 0.4975 - val_loss: 7.9382 - val_acc: 0.5021
